In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 50.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 74.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.2/224.2 kB 15.5 MB/s eta 0:00:00


In [38]:
import glob
import json

files = glob.glob(r"/content/drive/MyDrive/Coursework/Q_A/*.json")

In [39]:
texts = []
labels = []

def concat_data(path):
    with open(path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    for question, anss in data.items():
        for i in range(len(anss)):
            texts.append(question + '||||' + anss[i])
            labels.append(i)
            
for file in files:
    concat_data(file)

In [40]:
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from transformers import BertTokenizerFast, BertForSequenceClassification
from transformers import Trainer, TrainingArguments

class TextLabelDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.labels = labels
        self.tokenizer = tokenizer
        self.encodings = tokenizer(texts, max_length=64, return_tensors="pt", padding=True, truncation=True)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item



tokenizer = BertTokenizerFast.from_pretrained("cointegrated/rubert-tiny")

train_texts, test_texts, train_labels, test_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=0.25, random_state=42)

train_dataset = TextLabelDataset(train_texts, train_labels, tokenizer)
val_dataset = TextLabelDataset(val_texts, val_labels, tokenizer)
test_dataset = TextLabelDataset(test_texts, test_labels, tokenizer)

In [41]:
num_labels = len(set(labels))

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=num_labels)

# Set up training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    evaluation_strategy="epoch",
    save_strategy="epoch",
)

# Set up the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

# Train the model
trainer.train()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Epoch,Training Loss,Validation Loss
1,1.491300,1.544783
2,1.449800,1.511690
3,1.450700,1.486895


TrainOutput(global_step=666, training_loss=1.467863390753577, metrics={'train_runtime': 3021.3135, 'train_samples_per_second': 1.762, 'train_steps_per_second': 0.22, 'total_flos': 175153738089600.0, 'train_loss': 1.467863390753577, 'epoch': 3.0})

In [42]:
y_true = []
for i in test_dataset:
  y_true.append(i['labels'].detach().cpu().numpy())

In [43]:
import numpy
result = numpy.argmax(trainer.predict(test_dataset).predictions, axis=1)

In [51]:
from sklearn.metrics import f1_score, accuracy_score, recall_score

# Расчет метрик F1, accuracy и NDCG
f1 = f1_score(y_true, result, average='weighted')
accuracy = accuracy_score(y_true, result)
recall = recall_score(y_true, result, average='weighted')

print(f"F1 Score: {f1:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"Recall: {recall:.4f}")
from transformers import BertTokenizerFast

F1 Score: 0.3184
Accuracy: 0.4865
Recall: 0.4865


In [45]:
from transformers import BertConfig

config = BertConfig.from_json_file("/content/drive/MyDrive/Coursework/config.json")
own_model = BertForSequenceClassification.from_pretrained("/content/drive/MyDrive/Coursework/pytorch_model.bin", config=config)

Some weights of the model checkpoint at /content/drive/MyDrive/Coursework/pytorch_model.bin were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpo

In [46]:
own_model.num_labels = num_labels
own_model.classifier = torch.nn.Linear(in_features=768, out_features=num_labels)

In [47]:
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)
from transformers import AdamW, get_linear_schedule_with_warmup

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
own_model.to(device)

# Set up the optimizer and scheduler
optimizer = AdamW(own_model.parameters(), lr=5e-5)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=500, num_training_steps=len(train_loader) * 3)

# Training loop
num_epochs = 3

for epoch in range(num_epochs):
    own_model.train()
    for batch in train_loader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = own_model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()


    # Validation loop
    own_model.eval()
    total_val_loss = 0
    with torch.no_grad():
        for batch in val_loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = own_model(**batch)
            val_loss = outputs.loss
            total_val_loss += val_loss.item()

    avg_val_loss = total_val_loss / len(val_loader)
    print(f"Epoch: {epoch + 1}, Validation Loss: {avg_val_loss}")


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch: 1, Validation Loss: 1.5639895675955593
Epoch: 2, Validation Loss: 1.524721257992693
Epoch: 3, Validation Loss: 1.4877780204689182


In [48]:
own_model.eval()
preds = []
with torch.no_grad():
  for batch in val_loader:
      batch = {k: v.to(device) for k, v in batch.items()}
      outputs = own_model(**batch)
      preds.append(torch.argmax(outputs.logits, dim=1).detach().cpu().numpy())

In [49]:
y_pred = []
for pred in preds:
  for pr in pred:
    y_pred.append(pr)

In [50]:
from sklearn.metrics import f1_score, accuracy_score, recall_score

# Расчет метрик F1, accuracy и recall
f1 = f1_score(y_true, y_pred, average='weighted')
accuracy = accuracy_score(y_true, y_pred)
recall = recall_score(y_true, y_pred, average='weighted')

print(f"F1 Score: {f1:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"recall: {recall:.4f}")

F1 Score: 0.3184
Accuracy: 0.4865
recall: 0.4865
